<a href="https://colab.research.google.com/github/ghopper3/ChatGPT-Project/blob/main/Financial_Forecaster_1_0.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Financial Forecaster 1.0
This tool is designed to take historical financial statements and make a projection based on historical trends.

# High-Level Design

## Project Overview

*   Input: Spreadsheets with monthly corporate financial data (income statements).
*   Functionality: Parse the input data, analyze trends, and use a forecasting model to project future financials.
*  Output: Forecasted financial data for the specified number of future months or years.

## Components

* Data Loader: Module to load and parse spreadsheets.
* Data Preprocessor: Module to clean and prepare data for forecasting.
Forecaster: Core module that implements the forecasting model.
* Output Generator: Module to format and output the forecasted data.
* Forecasting Model: We will use a simple time series forecasting model such as ARIMA (AutoRegressive Integrated Moving Average), which is suitable for financial data forecasting. However, depending on the complexity of the data and the required accuracy, more sophisticated models like Prophet or LSTM neural networks could be considered.

# Project Setup

First, let's install the appropriate tools:

pandas for data manipulation.
openpyxl for reading Excel files.
statsmodels for ARIMA or another forecasting model.
matplotlib for plotting (optional).

In [ ]:
!pip install pandas openpyxl statsmodels matplotlib scikit-learn


# Implementation

Let's start with the coding part. We'll break it down into modules as outlined in the high-level design.

Data Loader Module (data_loader.py)
This module will contain functions to load the financial data from spreadsheets.

In [ ]:
import pandas as pd

def load_financial_data(filepath: str) -> pd.DataFrame:
    """
    Load monthly corporate financial data from an Excel spreadsheet.

    Args:
        filepath (str): Path to the spreadsheet.

    Returns:
        pd.DataFrame: Dataframe with financial metrics as rows and dates as columns.
    """
    # Load the spreadsheet, setting the first column as the index
    df = pd.read_excel(filepath, engine='openpyxl', index_col=0)

    # Parse the dates in the first row and set as column headers
    df.columns = pd.to_datetime(df.columns, format='%d/%b/%y')

    return df

# Adjust the filepath to point to the correct location of your spreadsheet
filepath = '/content/Income_Statement_2.xlsx'

# Load the data
financial_data = load_financial_data(filepath)
print(financial_data.head())  # Display the first few rows to verify the loading and transformation


               2017-09-30  2017-10-31  2017-11-30  2017-12-31    2018-01-31  \
US$ thousands                                                                 
Revenues          40850.7     41050.0     43630.2     44206.5  41016.666667   
COGS             -29157.2    -29442.5    -31836.5    -32486.5 -30147.250000   
Gross profit      11693.5     11607.5     11793.7     11720.0  10869.416667   
SG&A              -9577.0     -9687.0     -9510.3     -9481.6  -9500.000000   
EBITDA             2116.5      1920.5      2283.4      2238.4   1369.416667   

               2018-02-28  2018-03-31  2018-04-30    2018-05-31    2018-06-30  \
US$ thousands                                                                   
Revenues       43067.5000  43067.5000  43067.5000  45118.333333  45118.333333   
COGS          -31654.6125 -31654.6125 -31654.6125 -33161.975000 -33161.975000   
Gross profit   11412.8875  11412.8875  11412.8875  11956.358333  11956.358333   
SG&A           -9500.0000  -9500.0000  -9

Data Preprocessor Module (data_preprocessor.py)
This module will prepare the data for forecasting, handling any necessary cleaning or transformation.

In [ ]:
def preprocess_financial_data(df: pd.DataFrame) -> pd.DataFrame:
    """Preprocess the financial data for forecasting.

    Args:
        df (pd.DataFrame): Raw financial data.

    Returns:
        pd.DataFrame: Preprocessed financial data.
    """
    # Implement preprocessing steps as needed, e.g., handling missing values
    # Placeholder for preprocessing steps
    return df  # Return the preprocessed dataframe


Forecaster Module (forecaster.py)
This module will implement the forecasting logic using a chosen model.

In [ ]:
from sklearn.linear_model import LinearRegression
import numpy as np

def forecast_revenue_linear_regression(df: pd.DataFrame, periods: int) -> pd.DataFrame:
    # Filter DataFrame for the 'Revenues' metric
    revenue_df = df[df['Metric'] == 'Revenues'].copy()  # Create a copy to avoid modifying the original DataFrame
    revenue_df['Date'] = pd.to_datetime(revenue_df['Date'])
    revenue_df.sort_values(by='Date', inplace=True)

    # Prepare the features (X) and target (y)
    # Convert dates to ordinal numbers for linear regression
    X = np.array([d.toordinal() for d in revenue_df['Date']]).reshape(-1, 1)
    y = revenue_df['Value'].values

    # Fit the linear regression model
    model = LinearRegression()
    model.fit(X, y)

    # Generate dates for forecasting
    last_date = revenue_df['Date'].max()
    future_dates = [last_date + pd.DateOffset(months=m) for m in range(1, periods + 1)]
    future_dates_ordinal = np.array([d.toordinal() for d in future_dates]).reshape(-1, 1)

    # Forecast future revenues
    forecasted_values = model.predict(future_dates_ordinal)

    return pd.DataFrame({'Date': future_dates, 'Forecasted Revenue': forecasted_values})


Debugging:

In [ ]:
forecast_df = forecast_revenue_linear_regression(transformed_data, forecast_periods)
print(forecast_df)

         Date  Forecasted Revenue
0  2019-01-31        47924.930038
1  2019-02-28        48313.523926
2  2019-03-31        48743.752873
3  2019-04-30        49160.103466
4  2019-05-31        49590.332413
5  2019-06-30        50006.683007
6  2019-07-31        50436.911954
7  2019-08-31        50867.140901
8  2019-09-30        51283.491495
9  2019-10-31        51713.720441
10 2019-11-30        52130.071035
11 2019-12-31        52560.299982


Main Script (main.py)
This script will orchestrate the loading, preprocessing, forecasting, and output generation.

In [ ]:
def load_and_transform_financial_data(filepath: str) -> pd.DataFrame:
    df = pd.read_excel(filepath, engine='openpyxl', index_col="US$ thousands")
    df_long = df.stack().reset_index()
    df_long.columns = ['Metric', 'Date', 'Value']
    df_long['Date'] = pd.to_datetime(df_long['Date'])
    df_long['Value'] = pd.to_numeric(df_long['Value'], errors='coerce')
    return df_long

def forecast_metric(df: pd.DataFrame, metric: str, periods: int) -> pd.DataFrame:
    metric_df = df[df['Metric'] == metric]
    metric_df = metric_df.sort_values(by='Date')
    metric_df['Date'] = pd.to_datetime(metric_df['Date'])
    metric_df.set_index('Date', inplace=True)
    model = ARIMA(metric_df['Value'], order=(1, 1, 1))
    model_fit = model.fit()
    forecast = model_fit.forecast(steps=periods)
    return pd.DataFrame({f'Forecasted {metric}': forecast}, index=pd.date_range(start=metric_df.index[-1], periods=periods+1, closed='right'))

if __name__ == "__main__":
    filepath = '/content/Income_Statement_2.xlsx'
    forecast_periods = 12  # Forecast for the next 12 months

    # Load and transform the data
    transformed_data = load_and_transform_financial_data(filepath)

    # Forecast Revenues using Linear Regression
    forecast_df = forecast_revenue_linear_regression(transformed_data, forecast_periods)
    print(forecast_df)

    # Verify the length of the forecasted revenue series
forecasted_revenue_length = len(forecast_df['Forecasted Revenue'])
print(f"Length of forecasted revenue: {forecasted_revenue_length}")

# Compare with the expected periods to forecast
print(f"Expected periods to forecast: {periods_to_forecast}")

# Check if they match
if forecasted_revenue_length == periods_to_forecast:
    print("The lengths match.")
else:
    print(f"Mismatch in lengths. Expected {periods_to_forecast}, got {forecasted_revenue_length}.")


    # Save the forecasted DataFrame to a CSV file
    forecast_df.to_csv('/content/forecasted_revenues_linear_regression.csv', index=False)
    print("Forecasted data saved to /content/forecasted_revenues_linear_regression.csv")




         Date  Forecasted Revenue
0  2020-01-31        52990.491327
1  2020-02-29        53392.961012
2  2020-03-31        53823.187228
3  2020-04-30        54239.535178
4  2020-05-31        54669.761394
5  2020-06-30        55086.109344
6  2020-07-31        55516.335560
7  2020-08-31        55946.561775
8  2020-09-30        56362.909725
9  2020-10-31        56793.135941
10 2020-11-30        57209.483891
11 2020-12-31        57639.710107
Length of forecasted revenue: 12
Expected periods to forecast: 12
The lengths match.


Forecasting the rest of the income statement items will require additional calculations. We will use the following rules to forecast the rest of the items:

Forecast COGS: Calculate COGS as a percentage of revenue from historical data, then apply this percentage to the forecasted revenue.

Calculate Gross Profit: Subtract forecasted COGS from forecasted Revenues.

Forecast SG&A: Calculate SG&A as a percentage of revenue from historical data, then apply this percentage to the forecasted revenue.

Calculate EBITDA: Subtract forecasted SG&A from Gross Profit.

Determine Depreciation and Interest: Use the run rate from historical data.

Calculate EBIT: Subtract Depreciation from EBITDA.

Calculate Income Before Taxes: Subtract Interest from EBIT.

Forecast Taxes: Calculate Taxes as a percentage of revenue from historical data, then apply this percentage to the forecasted revenue.

Calculate Net Income: Subtract Taxes from Income Before Taxes.

Set Dividends Paid: Assume 0.0 as per your instruction.

In [ ]:
def forecast_financials(df: pd.DataFrame, forecasted_revenue_df: pd.DataFrame, periods_to_forecast: int) -> pd.DataFrame:
    forecasted_revenue_series = forecasted_revenue_df['Forecasted Revenue'].iloc[:periods_to_forecast]

    # Historical ratios
    historical_cogs_ratio = df.loc['COGS', :].sum() / df.loc['Revenues', :].sum()
    historical_sgna_ratio = df.loc['SG&A', :].sum() / df.loc['Revenues', :].sum()
    historical_taxes_ratio = df.loc['Taxes', :].sum() / df.loc['Revenues', :].sum()

    # Get the last historical depreciation value and calculate incremental increase
    last_historical_depreciation = df.loc['Depreciation', :].iloc[-1]
    forecasted_depreciation_values = [last_historical_depreciation * (1 + 0.0138/100) ** month for month in range(1, periods_to_forecast + 1)]

    # Other forecasts using historical ratios
    forecasted_cogs = forecasted_revenue_series * historical_cogs_ratio
    gross_profit = forecasted_revenue_series + forecasted_cogs
    forecasted_sgna = forecasted_revenue_series * historical_sgna_ratio
    ebitda = gross_profit + forecasted_sgna
    ebit = ebitda + forecasted_depreciation_values[-1]  # Use the dynamically calculated depreciation
    historical_interest = df.loc['Interest', :].mean()
    interest = historical_interest
    income_before_taxes = ebit + interest
    forecasted_taxes = forecasted_revenue_series * historical_taxes_ratio
    net_income = income_before_taxes + forecasted_taxes
    dividends_paid = pd.Series(0.0, index=forecasted_revenue_series.index)

    # Assemble forecasted financial statement
    forecasted_financials = pd.DataFrame({
        'Forecasted Revenue': forecasted_revenue_series,
        'Forecasted COGS': forecasted_cogs,
        'Gross Profit': gross_profit,
        'Forecasted SG&A': forecasted_sgna,
        'EBITDA': ebitda,
        'Depreciation': forecasted_depreciation_values,
        'EBIT': ebit,
        'Interest': interest,
        'Income Before Taxes': income_before_taxes,
        'Forecasted Taxes': forecasted_taxes,
        'Net Income': net_income,
        'Dividends Paid': dividends_paid,
    }, index=forecasted_revenue_series.index)

    return forecasted_financials

# Ensure this is calling the forecast_financials function and storing its return value
forecasted_financials_df = forecast_financials(financial_data, forecast_df, periods_to_forecast)

# Output the DataFrame as a CSV file
forecasted_financials_df.to_csv('/content/forecasted_financials.csv', index=True)

print("Forecasted financials saved to '/content/forecasted_financials.csv'")


Forecasted financials saved to '/content/forecasted_financials.csv'
